# Prepare Target data file
* **About 3 mins may be elapsed**
* Please refer to the 0.LookAt_RawData.ipynb if you see a process of handling a raw data file handled

In [2]:
import boto3
import os
import pandas as pd
import numpy as np

## Create Target Time Series File and Validation File
**IMPORTANT: Shift three days in the future because a weekly interval starts from Monday in Amazon Forecast.This dataset starts from Friday for an weekly interval**

In [11]:
data_dir = 'data'
train_file_name = 'train.csv'
features_file_name = 'features.csv'
store_file_name = 'stores.csv'
train_data = pd.read_csv(os.path.join(data_dir,train_file_name))
train_df = train_data.copy()
# Shift three days in the future 
train_df.Date = pd.to_datetime(train_df['Date']) + pd.DateOffset(days=3)
train_df = train_df.rename(columns={'Date':'date', 'Store':'item_id'})

In [12]:
temp_df = train_df.groupby(['item_id','date'])['Weekly_Sales'].sum()
# df2 = pd.DataFrame(temp_df, columns=['Store','Date','Weekly_Sales'])
stores_sales = pd.DataFrame(temp_df)
stores_sales.reset_index(inplace=True)
# print(stores_sales.groupby('Store').agg({'Date': ['min','max']}))
stores_sales = stores_sales.set_index('date')
stores_sales.head()

,item_id,Weekly_Sales
date,,
2010-02-08,1,1643690.90
2010-02-15,1,1641957.44
2010-02-22,1,1611968.17
2010-03-01,1,1409727.59
2010-03-08,1,1554806.68


In [13]:

stores_sales.item_id = stores_sales.item_id.astype(str)
stores_sales.index = pd.to_datetime(stores_sales.index, format = '%Y-%m-%d' )
# stores_sales.index = stores_sales.index.strftime('%Y-%m-%d %H:%M:%S')

#stores_sales.index = pd.to_datetime(stores_sales.index.strftime("yyyy-MM-dd hh:mm:ss"))
cols = ['Weekly_Sales', 'item_id']
stores_sales = stores_sales[cols]
stores_sales.head()

,Weekly_Sales,item_id
date,,
2010-02-08,1643690.90,1
2010-02-15,1641957.44,1
2010-02-22,1611968.17,1
2010-03-01,1409727.59,1
2010-03-08,1554806.68,1


## Split into target file and validation file

In [6]:
# end_train_date = '2011-10-03' # Non-inclusive
# end_val_date = '2012-01-02'

end_train_date = '2012-07-30' # Non-inclusive
end_val_date = '2012-10-29'

In [7]:
target_stores_sales = stores_sales[stores_sales.index < end_train_date]
validation_stores_sales = stores_sales[stores_sales.index >= end_train_date]
validation_stores_sales = stores_sales[stores_sales.index < end_val_date]


In [8]:
# With the data in a great state, save it off as a CSV
target_time_series_filename = "target_time_series.csv"
target_time_series_path = data_dir + "/" + target_time_series_filename
target_stores_sales.to_csv(target_time_series_path, header=False)

In [9]:
# With the data in a great state, save it off as a CSV
validation_time_series_filename = "validation_time_series.csv"
validation_time_series_path = data_dir + "/" + validation_time_series_filename
validation_stores_sales.to_csv(validation_time_series_path, header=False)

In [10]:
%store target_time_series_filename
%store target_time_series_path
%store target_stores_sales
%store validation_time_series_filename
%store validation_time_series_path
%store validation_stores_sales

Stored 'target_time_series_filename' (str)
Stored 'target_time_series_path' (str)
Stored 'target_stores_sales' (DataFrame)
Stored 'validation_time_series_filename' (str)
Stored 'validation_time_series_path' (str)
Stored 'validation_stores_sales' (DataFrame)
